# ToolNode in LangGraph: Specialized Tool Execution

A ToolNode in LangGraph is a specialized type of node designed specifically for executing tools. It provides a structured way to handle tool calls, manage tool results, and integrate external functionality into your graph workflows.

## Key Features:
- **Tool Execution**: Dedicated node for running tools
- **Result Management**: Handles tool outputs and errors
- **State Integration**: Seamlessly integrates tool results into state
- **Error Handling**: Built-in error management for tool failures

## ToolNode vs Regular Nodes:

### **Regular Node**
```python
def regular_node(state):
    """Regular node that might call tools"""
    # Some processing
    result = some_tool(state.get("input"))
    return {"processed": result}
```

### **ToolNode**
```python
from langgraph.prebuilt import ToolNode

# Create a ToolNode with specific tools
tool_node = ToolNode([weather_tool, database_tool, file_tool])

# ToolNode automatically handles:
# - Tool selection
# - Parameter extraction
# - Tool execution
# - Result formatting
```

## Creating ToolNodes:

### **Basic ToolNode**
```python
from langgraph.prebuilt import ToolNode
from langchain.tools import tool

@tool
def weather_tool(location: str) -> str:
    """Get weather for a location"""
    return f"Weather in {location}: Sunny, 25°C"

@tool
def calculator_tool(expression: str) -> str:
    """Calculate mathematical expression"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Invalid expression"

# Create ToolNode with tools
tool_node = ToolNode([weather_tool, calculator_tool])
```

### **ToolNode with Configuration**
```python
# ToolNode with custom configuration
tool_node = ToolNode(
    tools=[weather_tool, calculator_tool],
    name="my_tool_node",
    description="Node for executing various tools"
)
```

## ToolNode Integration:

### **Adding ToolNode to Graph**
```python
from langgraph import StateGraph

# Create graph
graph = StateGraph(MyState)

# Add ToolNode
graph.add_node("tools", tool_node)

# Add edges
graph.add_edge("classifier", "tools")
graph.add_edge("tools", "processor")
```

### **ToolNode with Conditional Routing**
```python
def route_after_tools(state):
    """Route based on tool execution results"""
    if state.get("tool_success"):
        return "success_handler"
    else:
        return "error_handler"

graph.add_conditional_edges(
    "tools",
    route_after_tools,
    {
        "success_handler": "process_results",
        "error_handler": "handle_errors"
    }
)
```

## ToolNode State Management:

### **Tool Selection**
```python
def select_tool(state):
    """Determine which tool to use"""
    query_type = state.get("query_type")
    
    if query_type == "weather":
        return "weather_tool"
    elif query_type == "calculation":
        return "calculator_tool"
    else:
        return "general_tool"
```

### **Tool Parameter Extraction**
```python
def extract_tool_params(state):
    """Extract parameters for tool execution"""
    return {
        "location": state.get("location"),
        "expression": state.get("expression"),
        "query": state.get("user_input")
    }
```

### **Tool Result Processing**
```python
def process_tool_results(state):
    """Process and format tool results"""
    tool_result = state.get("tool_result")
    
    return {
        "tool_success": tool_result.get("success", False),
        "tool_data": tool_result.get("data"),
        "tool_error": tool_result.get("error"),
        "execution_time": tool_result.get("execution_time")
    }
```

## Advanced ToolNode Patterns:

### **Multi-Tool ToolNode**
```python
# ToolNode with multiple related tools
data_tools = ToolNode([
    database_query_tool,
    data_validation_tool,
    data_transformation_tool
])

# ToolNode with different tool categories
utility_tools = ToolNode([
    email_tool,
    notification_tool,
    logging_tool
])
```

### **Conditional ToolNode**
```python
def conditional_tool_node(state):
    """ToolNode that selects tools based on state"""
    available_tools = []
    
    if state.get("needs_weather"):
        available_tools.append(weather_tool)
    
    if state.get("needs_calculation"):
        available_tools.append(calculator_tool)
    
    if state.get("needs_database"):
        available_tools.append(database_tool)
    
    return ToolNode(available_tools)
```

### **ToolNode with Error Recovery**
```python
def resilient_tool_node(state):
    """ToolNode with error recovery"""
    try:
        # Primary tool execution
        result = primary_tool(state.get("input"))
        return {"tool_result": result, "status": "success"}
    
    except Exception as e:
        # Fallback to secondary tool
        try:
            result = fallback_tool(state.get("input"))
            return {"tool_result": result, "status": "fallback"}
        except Exception as e2:
            return {"error": str(e2), "status": "failed"}
```

## ToolNode Best Practices:

### **1. Tool Organization**
- Group related tools together
- Use descriptive tool names
- Document tool purposes clearly
- Keep tools focused and single-purpose

### **2. Error Handling**
- Always handle tool failures
- Provide meaningful error messages
- Implement fallback mechanisms
- Log tool execution details

### **3. Performance**
- Use async tools when possible
- Implement tool caching
- Set appropriate timeouts
- Monitor tool execution times

### **4. Testing**
- Test each tool independently
- Mock external dependencies
- Test error scenarios
- Validate tool outputs

## Common ToolNode Use Cases:

### **Data Processing Pipeline**
```python
# ToolNode for data processing
data_pipeline_tools = ToolNode([
    data_extraction_tool,
    data_validation_tool,
    data_transformation_tool,
    data_loading_tool
])
```

### **API Integration**
```python
# ToolNode for API integration
api_tools = ToolNode([
    weather_api_tool,
    news_api_tool,
    stock_api_tool
])
```

### **User Interaction**
```python
# ToolNode for user interactions
interaction_tools = ToolNode([
    email_tool,
    sms_tool,
    notification_tool
])
```

## ToolNode State Structure:

### **Input State**
```python
class ToolNodeInput(TypedDict):
    tool_name: str
    tool_params: Dict[str, Any]
    execution_context: Dict[str, Any]
```

### **Output State**
```python
class ToolNodeOutput(TypedDict):
    tool_result: Any
    tool_success: bool
    tool_error: Optional[str]
    execution_time: float
    tool_name: str
```

## ToolNode vs Manual Tool Calls:

### **Manual Tool Calls**
```python
def manual_tool_node(state):
    """Manual tool execution"""
    tool_name = state.get("tool_name")
    params = state.get("tool_params", {})
    
    if tool_name == "weather":
        result = weather_tool(params.get("location"))
    elif tool_name == "calculator":
        result = calculator_tool(params.get("expression"))
    # ... more manual handling
    
    return {"tool_result": result}
```

### **ToolNode Approach**
```python
# Much simpler with ToolNode
tool_node = ToolNode([weather_tool, calculator_tool])
# ToolNode handles tool selection and execution automatically
```

## Benefits:
- **Simplified Tool Management**: Centralized tool execution
- **Automatic Tool Selection**: Smart tool routing
- **Built-in Error Handling**: Robust error management
- **State Integration**: Seamless state updates
- **Reusability**: Easy to reuse across graphs

## Tips for Success:
- Use ToolNode for tool-heavy workflows
- Group related tools together
- Implement proper error handling
- Test tools independently
- Monitor tool performance
- Document tool interfaces
- Consider tool dependencies
- Plan for tool failures
